# Métodos Computacionles para las Políticas Públicas
# Proyecto Final - Análisis Envolvente de Datos
## Santiago Ortiz Ortiz

Importamos las librerias necesarias para desarrollar Web Scraping a la página.

In [25]:
import pandas as pd
import re
import requests
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains

**Para ingresar al panel principal.**

La página web a la que le vamos a hacer WS es Sistema de Gestión Hospitalaria (SIHO).
Url principal: https://prestadores.minsalud.gov.co/siho/
Está es la URL principal. Sin embargo, cuando uno quiere examinar la página se da cuenta que tiene unas macro que no se muestran de forma tan simple. Por lo tanto, ingresamos a un link secundario que habilita la macro: https://prestadores.minsalud.gov.co/siho/work.aspx

Una vez acá, el software nos habilita un usuario de invitado. Este ya está digitado, solo hay que darle click en ingresar.

In [62]:
# Creamos una variable con la URL de la página.
url = "https://prestadores.minsalud.gov.co/siho/work.aspx"

# Con ayuda del WebDriver de Chrome vamos a abrir la página.
browser = webdriver.Chrome(executable_path=r"C:\Users\tatoo\Desktop\PhantomJS\Chrome\chromedriver.exe")
# A la hora de hacer Web Scraping es importante dejar un periodo de tiempo relativamente largo (algo que aparente ser humano)
time.sleep(5)
# Con el .get(url), el programa sabe que tiene que ingresar a esa URL
browser.get(url)
time.sleep(5)
# Le pedimos al programa que encuentre un elemento en el código HTML que se llame "btnIngresar" y que le dé CLICK
boton_submit = browser.find_element_by_name("btnIngresar").click()
time.sleep(4)

**Para que Python sepa que esta en otra pestaña**
https://stackoverflow.com/questions/17676036/python-webdriver-to-handle-pop-up-browser-windows-which-is-not-an-alert

In [63]:
# Fijamos una página principal llamada MAIN
main_window_handle = None
while not main_window_handle:
    main_window_handle = browser.current_window_handle
browser.find_element_by_link_text('Capacidad Instalada').click()
signin_window_handle = None
while not signin_window_handle:
    for handle in browser.window_handles:
        if handle != main_window_handle:
            signin_window_handle = handle
            break
browser.switch_to.window(signin_window_handle)

In [64]:
browser.page_source # Para comprobar que tenga la nueva forma

'<html xmlns="http://www.w3.org/1999/xhtml"><head><title>\n\tCapacidad Instalada\n</title><link href="../includes/appestilos.css" rel="stylesheet"></head>\n<body bgcolor="#ffffff">\n    <form name="aspnetForm" method="post" action="./capacidadinstaladanacional.aspx?pageTitle=Capacidad+Instalada&amp;pageHlp=%2fSIHO%2fayudas%2finformes%2fcapacidad.pdf" id="aspnetForm">\n<input type="hidden" name="__VIEWSTATE" id="__VIEWSTATE" value="/wEPDwULLTIwNDI2NTY5MjEPZBYCZg9kFgICAw9kFggCDw8PFgIeC05hdmlnYXRlVXJsBSMvU0lITy9heXVkYXMvaW5mb3JtZXMvY2FwYWNpZGFkLnBkZmRkAhEPDxYCHgRUZXh0BRNDYXBhY2lkYWQgSW5zdGFsYWRhZGQCEw9kFgQCAQ9kFgJmD2QWAmYPZBYEAgEPEGQQFRIEMjAwMgQyMDAzBDIwMDQEMjAwNQQyMDA2BDIwMDcEMjAwOAQyMDA5BDIwMTAEMjAxMQQyMDEyBDIwMTMEMjAxNAQyMDE1BDIwMTYEMjAxNwQyMDE4BDIwMTkVEgQyMDAyBDIwMDMEMjAwNAQyMDA1BDIwMDYEMjAwNwQyMDA4BDIwMDkEMjAxMAQyMDExBDIwMTIEMjAxMwQyMDE0BDIwMTUEMjAxNgQyMDE3BDIwMTgEMjAxORQrAxJnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dkZAIDDxAPFgYeDURhdGFUZXh0RmllbGQFBm5vbWJyZR4ORGF0YVZhbHVlRmllbGQFBmNvZGlnbx4LXyFEYXRhQ

Lo que hacen los dos códigos siguientes es: 
- Encontrar las casillas año y departamento. Como ambas son elementos de multiple opción este desplegará unos valores.
- Año: 2002 - 2019 (Sin embargo, no hay datos para el 2019)
- Depto: Amazonas - Vaupés (+Bogotá DC.)

In [65]:
# El espacio del año tiene un nombre como el siguiente:
cod_año = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:ddano")
for option in cod_año.find_elements_by_tag_name('option'):
    if option.text == '2003':
        option.click() 
        break

In [91]:
# El espacio del departamento tiene un nombre como el siguiente:
cod_depto = browser.find_element_by_name("_ctl0:ContentPlaceHolder1:dddepa_codigo")
for option in cod_depto.find_elements_by_tag_name('option'):
    if option.text == 'Caldas':
        option.click() # select() in earlier versions of webdriver
        break

In [92]:
# Después de seleccionar los departamentoS Y LOS AÑOS LE DAMOS CLICK a Buscar
boton_buscar = browser.find_element_by_name("_ctl0:ibBuscarFtr")
boton_buscar.click()

In [93]:
html = browser.execute_script("return document.getElementsByTagName('html')[0].innerHTML")   
soup = BeautifulSoup(html, "html.parser")

In [94]:
re.findall('<td align="center">(\d*)</td><td>(.*)</td><td align="right">(\d*)</td><td align="right">[\d,]*</td><td align="right">[\d,]*</td>', str(soup))

[('1', 'Camas de hospitalización', '678'),
 ('2', 'Camas de hospitalización', '167'),
 ('3', 'Camas de hospitalización', '287'),
 ('1', 'Camas de observación', '74'),
 ('2', 'Camas de observación', '39'),
 ('3', 'Camas de observación', '10'),
 ('1', 'Consultorios de consulta externa', '165'),
 ('2', 'Consultorios de consulta externa', '44'),
 ('3', 'Consultorios de consulta externa', '15'),
 ('1', 'Consultorios en el servicio de urgencias', '33'),
 ('2', 'Consultorios en el servicio de urgencias', '14'),
 ('3', 'Consultorios en el servicio de urgencias', '4'),
 ('1', 'Salas de quirófanos', '12'),
 ('2', 'Salas de quirófanos', '6'),
 ('3', 'Salas de quirófanos', '5'),
 ('1', 'Mesas de partos', '35'),
 ('2', 'Mesas de partos', '7'),
 ('3', 'Mesas de partos', '0'),
 ('1', 'Número de unidades de odontología', '79'),
 ('2', 'Número de unidades de odontología', '13'),
 ('3', 'Número de unidades de odontología', '0')]

In [95]:
agregar = re.findall('<td align="center">(\d*)</td><td>(.*)</td><td align="right">(\d*)</td><td align="right">[\d,]*</td><td align="right">[\d,]*</td>', str(soup))

**Ideas para la base de datos:**
- https://thispointer.com/python-pandas-how-to-add-rows-in-a-dataframe-using-dataframe-append-loc-iloc/

In [84]:
df_base = pd.DataFrame(columns=['Nivel', 'Concepto', 'Cantidad', 'Departamento', 'Año', 'Clase'])
df_base

,Nivel,Concepto,Cantidad,Departamento,Año,Clase


In [96]:
agregar_base = pd.DataFrame(agregar, columns = ['Nivel', 'Concepto', 'Cantidad']) 
agregar_base

,Nivel,Concepto,Cantidad
0,1,Camas de hospitalización,678
1,2,Camas de hospitalización,167
2,3,Camas de hospitalización,287
3,1,Camas de observación,74
4,2,Camas de observación,39
5,3,Camas de observación,10
6,1,Consultorios de consulta externa,165
7,2,Consultorios de consulta externa,44
8,3,Consultorios de consulta externa,15
9,1,Consultorios en el servicio de urgencias,33


In [97]:
len(agregar_base["Nivel"]) # Esta siempre aparece, para ver cuantos datos son 

21

In [98]:
depto = []
for i in range(len(agregar_base["Nivel"])):
    depto.append("Caldas")
depto

['Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas',
 'Caldas']

In [99]:
agregar_base["Departamento"] = depto
agregar_base

,Nivel,Concepto,Cantidad,Departamento
0,1,Camas de hospitalización,678,Caldas
1,2,Camas de hospitalización,167,Caldas
2,3,Camas de hospitalización,287,Caldas
3,1,Camas de observación,74,Caldas
4,2,Camas de observación,39,Caldas
5,3,Camas de observación,10,Caldas
6,1,Consultorios de consulta externa,165,Caldas
7,2,Consultorios de consulta externa,44,Caldas
8,3,Consultorios de consulta externa,15,Caldas
9,1,Consultorios en el servicio de urgencias,33,Caldas


In [100]:
df_base = df_base.append(agregar_base, ignore_index=True, sort=False)

In [101]:
df_base

,Nivel,Concepto,Cantidad,Departamento,Año,Clase
0,1,Camas de hospitalización,253,Bolívar,NaN,NaN
1,2,Camas de hospitalización,459,Bolívar,NaN,NaN
2,1,Camas de observación,199,Bolívar,NaN,NaN
3,2,Camas de observación,65,Bolívar,NaN,NaN
4,1,Consultorios de consulta externa,227,Bolívar,NaN,NaN
5,2,Consultorios de consulta externa,45,Bolívar,NaN,NaN
6,1,Consultorios en el servicio de urgencias,59,Bolívar,NaN,NaN
7,2,Consultorios en el servicio de urgencias,11,Bolívar,NaN,NaN
8,1,Salas de quirófanos,16,Bolívar,NaN,NaN
9,2,Salas de quirófanos,12,Bolívar,NaN,NaN


In [60]:
base = base.append(prueba, columns = ['Nivel', 'Concepto', 'Cantidad']) 

TypeError: append() got an unexpected keyword argument 'columns'

Hasta acá funciona melo!, pero hay que pensar en una forma de hacer que Python sepa que va a trabajar en la otra ventana
- Revisar: https://stackoverflow.com/questions/32858434/switch-to-popup-in-python-using-selenium

In [53]:
vacia = []
if len(vacia) == 0:
    vacia = [(0,0,0,0)]
vacia

[(0, 0, 0, 0)]